In [1]:
import sympy as sym
from sympy.functions import Heaviside, sin, cos, sqrt
#from sympy.vector import *
sym.init_printing()

# This IPython magic generates a table with version information
#https://github.com/jrjohansson/version_information
#%load_ext version_information
#%version_information sympy

In [2]:
from sympy import I, Matrix, symbols
from sympy.physics.quantum import TensorProduct,Dagger


def spre(m):
    return TensorProduct(sym.eye(m.shape[0]),m)


def spost(m):
    return TensorProduct(m.T,sym.eye(m.shape[0]))

def collapse(c):
    temp = Dagger(c)*c
    return 2*spre(c)*spost(Dagger(c))-spre(temp)-spost(temp)

s13=Matrix([[0,0,1],[0,0,0],[0,0,0]])
s23=Matrix([[0,0,0],[0,0,1],[0,0,0]])
s12=Matrix([[0,1,0],[0,0,0],[0,0,0]])

s31=s13.T
s32=s23.T
s21=s12.T

s11 = s12*s21
s22 = s21*s12
s33 = s31*s13



In [3]:
delta2,delta3 = sym.symbols('delta_2 delta_3',real=True)
gamma13,gamma23,gamma2d,gamma3d,nbath,gammam = sym.symbols('gamma_13 gamma_23 gamma_2d gamma_3d n_b gamma_m', real=True, negative=False)
omegao,omegam,omegasig = sym.symbols('Omega_o Omega_m Omega_sb')
sigmm,sigmo = sym.symbols('sigma_m sigma_o')

In [4]:
H =  omegam*s21 + omegao*s32 + omegasig*s31
H = H + Dagger(H)
H = H + delta2*s22 + delta3*s33
LH = -I*spre(H)+I*spost(H) 
L21 = gammam/2*(nbath+1)*collapse(s12)
L12 = gammam/2*(nbath)*collapse(s21)
L32 = gamma23/2*collapse(s23)
L31 = gamma13/2*collapse(s13)
L22 = gamma2d/2*collapse(s22)
L33 = gamma3d/2*collapse(s33)

L = LH + L21 + L12 + L32 + L31 + L22 + L33

#becausetrace rho = 1 we can replace first row
L = L.row_insert(0,Matrix([[1,0,0,0,1,0,0,0,1]]))
L.row_del(1)


L

⎡   1                      0                                    0             
⎢                                                                             
⎢                 γ_2d   γₘ⋅n_b   γₘ⋅(n_b + 1)                   __           
⎢ -ⅈ⋅Ωₘ   -ⅈ⋅δ₂ - ──── - ────── - ────────────                -ⅈ⋅Ωₒ           
⎢                  2       2           2                                      
⎢                                                                             
⎢                                                       γ₁₃   γ₂₃   γ_3d   γₘ⋅
⎢-ⅈ⋅Ω_sb                 -ⅈ⋅Ωₒ                  -ⅈ⋅δ₃ - ─── - ─── - ──── - ───
⎢                                                        2     2     2       2
⎢                                                                             
⎢   __                                                                        
⎢ ⅈ⋅Ωₘ                     0                                    0             
⎢                                                   

In [5]:
L[:,0:3] #because the matrix gets clipped

⎡   1                      0                                    0             
⎢                                                                             
⎢                 γ_2d   γₘ⋅n_b   γₘ⋅(n_b + 1)                   __           
⎢ -ⅈ⋅Ωₘ   -ⅈ⋅δ₂ - ──── - ────── - ────────────                -ⅈ⋅Ωₒ           
⎢                  2       2           2                                      
⎢                                                                             
⎢                                                       γ₁₃   γ₂₃   γ_3d   γₘ⋅
⎢-ⅈ⋅Ω_sb                 -ⅈ⋅Ωₒ                  -ⅈ⋅δ₃ - ─── - ─── - ──── - ───
⎢                                                        2     2     2       2
⎢                                                                             
⎢   __                                                                        
⎢ ⅈ⋅Ωₘ                     0                                    0             
⎢                                                   

In [6]:
L[:,3:6]

⎡                 0                         1                                 
⎢                                                                             
⎢                 0                       ⅈ⋅Ωₘ                                
⎢                                                                             
⎢                 0                         0                                ⅈ
⎢                                                                             
⎢       γ_2d   γₘ⋅n_b   γₘ⋅(n_b + 1)         __                               
⎢ⅈ⋅δ₂ - ──── - ────── - ────────────      -ⅈ⋅Ωₘ                            -ⅈ⋅
⎢        2       2           2                                                
⎢                                                                             
⎢                                                                             
⎢               -ⅈ⋅Ωₘ                 -γₘ⋅(n_b + 1)                         -ⅈ
⎢                                                   

In [7]:
L[:,6:]

⎡               0                                            0                
⎢                                                                             
⎢               0                                         ⅈ⋅Ω_sb              
⎢                                                                             
⎢               0                                            0                
⎢                                                                             
⎢              ⅈ⋅Ωₒ                                          0                
⎢                                                                             
⎢               0                                          ⅈ⋅Ωₒ               
⎢                                                                             
⎢               0                                            0                
⎢                                                                             
⎢       γ₁₃   γ₂₃   γ_3d   γₘ⋅n_b                   

In [8]:
rho11,rho12,rho13,rho21,rho22,rho23,rho31,rho32,rho33 = sym.symbols('rho_11 rho_12 rho_13 rho_21 rho_22 rho_23 rho_31 rho_32 rho_33')
rho = Matrix([[rho11,rho21,rho31],[rho12,rho22,rho32],[rho13,rho23,rho33]])
#rho = 1*rho.T
#rho[:]
rhoflat = 1*rho.T 
rhoflat = rhoflat[:]
rhoflat[6]

In [9]:
from sympy.printing import print_ccode
Lflat = 1*L.T
Lflat = Lflat[:]
print_ccode(Lflat)

// Not supported in C:
// conjugate
// conjugate
// conjugate
[1, -I*Omega_m, -I*Omega_sb, I*conjugate(Omega_m), gamma_m*n_b, 0, I*conjugate(Omega_sb), 0, 0, 0, -I*delta_2 - 1.0/2.0*gamma_2d - 1.0/2.0*gamma_m*n_b - 1.0/2.0*gamma_m*(n_b + 1), -I*Omega_o, 0, I*conjugate(Omega_m), 0, 0, I*conjugate(Omega_sb), 0, 0, -I*conjugate(Omega_o), -I*delta_3 - 1.0/2.0*gamma_13 - 1.0/2.0*gamma_23 - 1.0/2.0*gamma_3d - 1.0/2.0*gamma_m*n_b, 0, 0, I*conjugate(Omega_m), 0, 0, I*conjugate(Omega_sb), 0, 0, 0, I*delta_2 - 1.0/2.0*gamma_2d - 1.0/2.0*gamma_m*n_b - 1.0/2.0*gamma_m*(n_b + 1), -I*Omega_m, -I*Omega_sb, I*conjugate(Omega_o), 0, 0, 1, I*Omega_m, 0, -I*conjugate(Omega_m), -gamma_m*(n_b + 1), -I*Omega_o, 0, I*conjugate(Omega_o), 0, 0, 0, I*Omega_m, -I*conjugate(Omega_sb), -I*conjugate(Omega_o), I*delta_2 - I*delta_3 - 1.0/2.0*gamma_13 - 1.0/2.0*gamma_23 - 1.0/2.0*gamma_2d - 1.0/2.0*gamma_3d - 1.0/2.0*gamma_m*(n_b + 1), 0, 0, I*conjugate(Omega_o), 0, 0, 0, I*Omega_o, 0, 0, I*delta_3 - 1.0/2.0*gamma_13